In [2]:
import os
os.chdir('../')

In [72]:
from extract import *
import string
import re
from collections import defaultdict
from math import log

In [4]:
%load_ext autoreload
%autoreload 2

In [114]:
def update_url_scores(old: dict[str, float], new: dict[str, float]):
    for url, score in new.items():
        if url in old:
            old[url] += score
        else:
            old[url] = score
    return old

class SearchEngine:
    def __init__(self, k1: float = 1.5, b: float = 0.75):
        self._index: dict[str, dict[str, int]] = defaultdict(lambda: defaultdict(int))
        self._documents: dict[str, str] = {}
        self.k1 = k1
        self.b = b

    @property
    def posts(self) -> list[str]:
        return list(self._documents.keys())

    @property
    def number_of_documents(self) -> int:
        return len(self._documents)

    @property
    def avdl(self) -> float:
        # todo: refactor this. it can be slow to compute it every time. compute it once and cache it
        return sum(len(d) for d in self._documents.values()) / len(self._documents)

    def idf(self, kw: str) -> float:
        N = self.number_of_documents
        n_kw = len(self.get_urls(kw))
        return log((N - n_kw + 0.5) / (n_kw + 0.5) + 1)

    def bm25(self, kw: str) -> dict[str, float]:
        result = {}
        idf_score = self.idf(kw)
        avdl = self.avdl
        for url, freq in self.get_urls(kw).items():
            numerator = freq * (self.k1 + 1)
            denominator = freq + self.k1 * (
                1 - self.b + self.b * len(self._documents[url]) / avdl
            )
            result[url] = idf_score * numerator / denominator
        return result

    def search(self, query: str, top_n=10) -> dict[str, float]:
        keywords = normalize_string(query).split(" ")
        url_scores: dict[str, float] = {}
        for kw in keywords:
            kw_urls_score = self.bm25(kw)
            url_scores = update_url_scores(url_scores, kw_urls_score)
        
        url_scores = sorted(url_scores.items(), key=lambda x: x[1], reverse=True)[:10]
        return url_scores

    def index(self, url: str, content: str) -> None:
        self._documents[url] = content
        words = content.split()
        # words = normalize_string(content).split(" ")
        for word in words:
            self._index[word][url] += 1

    def index_entity(self, entity):
        entity_id = get_entity_id(entity)
        raw_content = entity['meta']
        raw_df_content = get_text_from_df_v2(entity['frame'])
        search_content = normalize_string(entity['meta']) + '\n' + normalize_string(raw_df_content)
        self.index(entity_id, search_content)

    def bulk_index(self, documents: list[tuple[str, str]]):
        for url, content in documents:
            self.index(url, content)

    def get_urls(self, keyword: str) -> dict[str, int]:
        keyword = normalize_string(keyword)
        return self._index[keyword]

In [7]:
entities = read_entities('documents.json')
entities[0]

{'url': 'https://cbr.ru/banking_sector/credit/FullCoList/',
 'frame':        0              1       2              3                          4  \
 0                                                                           
 1      1                   2896  1021500000147            ПАО АКБ «1Банк»   
 2      2                   2306  1027700024560   АКБ «Абсолют Банк» (ПАО)   
 3      3                   2879  1027700367507         ПАО АКБ «АВАНГАРД»   
 4      4                    415  1231600067654            АО Банк «Аверс»   
 ..   ...            ...     ...            ...                        ...   
 671  671                   3467  1067711004437           АО КБ «ЮНИСТРИМ»   
 672  672  Расчетная НКО  3549-К  1247700338049  РНКО «Юнона Финанс» (ООО)   
 673  673  Расчетная НКО  3541-К  1217700542344           ООО РНКО «ЮСиЭс»   
 674  674                   3027  1077711000091           АО «Яндекс Банк»   
 675  675                   2564  1027600000075   ИКБР «ЯРИНТЕРБАНК» (ООО

In [8]:
test_entity = entities[0]

In [9]:
test_entity['meta']

'    Список кредитных организаций, зарегистрированных на территории Российской Федерации по состоянию на 07.06.2024 | Банк России'

In [29]:
def normalize_whitespace(s):
    s = re.sub("\s+", " ", s)
    return s.strip()

def normalize_whitespace_v2(s):
    return ' '.join(s.split())

def normalize_punctuation(input_string: str) -> str:
    return re.sub(r"[^\w\s]+", ' ', input_string)

def normalize_case(s):
    return s.lower()

def normalize_string(s):
    pipeline = [
        normalize_case,
        normalize_whitespace,
        normalize_whitespace_v2,
        normalize_punctuation,
    ]
    for op in pipeline:
        s = op(s)
    return s

In [97]:
def hash_df(df):
    return pd.util.hash_pandas_object(df, index=True).sum()

def get_entity_id(entity):
    return f'{entity["url"]}@{hash_df(entity["frame"])}'

def get_url_from_entity_id(entity_id):
    return '@'.join(entity_id.split('@')[:-1])

def get_text_from_df(df):
    texts = []
    for i in test_entity['frame'].values:
        for j in i:
            if get_type(j) == 'str':
                texts.append(j)
    text_doc = '\n'.join(texts)
    return text_doc

def get_text_from_df_v2(df):
    return normalize_string(df.to_string())


In [78]:
from tqdm import tqdm

In [88]:
%%timeit

get_text_from_df(entities[0]['frame'])

403 ms ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [99]:
%%timeit

get_text_from_df_v2(entities[0]['frame'])

17 ms ± 431 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [115]:
se = SearchEngine()

for entity in tqdm(entities):
    se.index_entity(entity)

100%|████████████████████████████████████████| 141/141 [00:00<00:00, 319.97it/s]


In [112]:
se = SearchEngine()

for entity in tqdm(entities):
    entity_id = get_entity_id(entity)
    raw_content = entity['meta']
    raw_df_content = get_text_from_df_v2(entity['frame'])
    search_content = normalize_string(entity['meta']) + '\n' + normalize_string(raw_df_content)
    se.index(entity_id, search_content)

100%|████████████████████████████████████████| 141/141 [00:00<00:00, 299.14it/s]


In [116]:
se.search('банк')

[('https://cbr.ru/registries/nps/oper_zip/@8801460840335433866',
  0.148376227083583),
 ('https://cbr.ru/banking_sector/credit/FullCoList/@-7638025034285683774',
  0.1448388839655293),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=460000022@-7402489107672318118',
  0.13410096480527095),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=450000203@-5301193403033226121',
  0.1330282201650436),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=350000004@-7024088650001845899',
  0.12914335763387438),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=10000005@599769350099677401',
  0.1260141945050062),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=450039265@-4301329512981266293',
  0.12596213036484183),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=980000021@2245106942322526861',
  0.12559888251845971),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=400000023@974109305061869944',
  0.1254954819093079),
 ('https://cbr.ru/banking_sector/credit/coinfo/?id=450000373@3757071

In [63]:
test_entity['meta']

'    Список кредитных организаций, зарегистрированных на территории Российской Федерации по состоянию на 07.06.2024 | Банк России'

In [24]:
test_entity['frame']

,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
1,1,,2896,1021500000147,ПАО АКБ «1Банк»,ПАО,10.06.1994,ОТЗ,"362040, Республика Северная Осетия-Алания, г.В..."
2,2,,2306,1027700024560,АКБ «Абсолют Банк» (ПАО),ПАО,22.04.1993,,"127051, г. Москва, Цветной бульвар, д. 18"
3,3,,2879,1027700367507,ПАО АКБ «АВАНГАРД»,ПАО,09.06.1994,,"119180, г. Москва, ул. Большая Якиманка, д.1"
4,4,,415,1231600067654,АО Банк «Аверс»,НПАО,25.09.1990,,"420111, г.Казань, ул.Мусы Джалиля, д.3"
...,...,...,...,...,...,...,...,...,...
671,671,,3467,1067711004437,АО КБ «ЮНИСТРИМ»,НПАО,31.05.2006,,"127083, г. Москва, ул. Верхняя Масловка, д. 20..."
672,672,Расчетная НКО,3549-К,1247700338049,РНКО «Юнона Финанс» (ООО),ООО (Паевое),24.04.2024,,"127015, Российская Федерация, город Москва, ул..."
673,673,Расчетная НКО,3541-К,1217700542344,ООО РНКО «ЮСиЭс»,ООО (Паевое),12.11.2021,АНН,"117449, г. Москва, ул. Новочерёмушкинская, д.10"
674,674,,3027,1077711000091,АО «Яндекс Банк»,НПАО,04.08.1994,,"115035, г. Москва, ул. Садовническая, д. 82 ст..."
